In [11]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = SparkSession.builder.appName("SparkML Train Test").getOrCreate()
irisDF= spark.read.option("header","true")\
                        .option("inferSchema","True")\
                        .csv("datasets/iris-dataset.txt")
#irisDF.show(5)
strIndexer = StringIndexer(inputCol='class',outputCol='label')
irisDF = strIndexer.fit(irisDF).transform(irisDF)
classesDF = irisDF.select("class").groupBy("class").count()
classesDF.show()
#print(irisDF.columns[0:4])
vec =  VectorAssembler(inputCols=irisDF.columns[0:4],outputCol='features')
irisDF = vec.transform(irisDF)
irisDF = irisDF.select('features','label')
#irisDF.show()

trainDF, testDF = irisDF.randomSplit([0.75,0.25],seed=1112)

dtClassifier = MultilayerPerceptronClassifier(layers=[4,2,3])
model = dtClassifier.fit(trainDF)
#print("Model created learned !!! :)")

resultDF = model.transform(testDF) #Prediction
#resultDF.show(100)

eva = MulticlassClassificationEvaluator(metricName='accuracy')
accuracy = eva.evaluate(resultDF)
print("Test Accuracy : ",accuracy)

+---------------+-----+
|          class|count|
+---------------+-----+
| Iris-virginica|   50|
|    Iris-setosa|   50|
|Iris-versicolor|   50|
+---------------+-----+

Test Accuracy :  0.975609756097561
